In [2]:
# Imports

import os
import json
import tqdm
import yaml

import torch
import numpy as np
import matplotlib.pyplot as plt

from omegaconf import OmegaConf
from pathlib import Path
from tqdm.auto import tqdm

/home/denis/miniconda3/envs/dal-toolbox/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
%%bash
mkdir -p results
fusermount -uz results
sshfs compute.ies:/mnt/work/dhuseljic/results/hyperparameters/ results
ls results

lr0.001_wd0.0005
lr0.001_wd0.005
lr0.001_wd0.05
lr0.01_wd0.0005
lr0.01_wd0.005
lr0.01_wd0.05
lr0.1_wd0.0005
lr0.1_wd0.005
lr0.1_wd0.05


In [8]:

def load_json(json_file):
    with open(json_file, 'r') as f:
        data = json.load(f)
    return data

def load_results(path):
    path = Path(path)
    assert path.is_dir(), 'Path does not exist.'
    exp_json = path / 'results.json'
    results = load_json(exp_json)
    return results

def load_args(path):
    path = Path(path)
    assert path.is_dir(), 'Path does not exist.'
    exp_cfg = path / '.hydra' / 'config.yaml'
    cfg =  OmegaConf.load(exp_cfg)
    return cfg

def load_checkpoint(path, final=True):
    path = Path(path)
    assert path.is_dir(), 'Path does not exist.'
    if final:
        exp_pth = path / 'model_final.pth'
    else:
        exp_pth = path / 'checkpoint.pth'
    checkpoint = torch.load(exp_pth)
    return checkpoint

def get_experiments(result_path, glob_pattern, train_results=False):
    # Aggregate results over multiple glob pattern such as seeds
    experiments = []
    for exp_path in result_path.glob(glob_pattern):
        d = load_results(exp_path)
        experiments.append(d)
    assert len(experiments) != 0, f'No experiments found for {result_path}.'
    return experiments

path = f'results/lr0.001_wd0.0005/'
exp_results = get_experiments(Path(path), 'seed1')
# metrics = get_metric_values(exp_results)
print(exp_results[0].keys())
# print(len(exp_results[0]['results']['train_history']))
# print(exp_results[0]['results']['test_stats'].keys())

dict_keys(['cycle0', 'cycle1', 'cycle2', 'cycle3', 'cycle4', 'cycle5', 'cycle6', 'cycle7', 'cycle8', 'cycle9', 'cycle10', 'cycle11', 'cycle12', 'cycle13', 'cycle14', 'cycle15', 'cycle16', 'cycle17', 'cycle18', 'cycle19', 'cycle20', 'cycle21', 'cycle22', 'cycle23', 'cycle24', 'cycle25', 'cycle26', 'cycle27', 'cycle28', 'cycle29', 'cycle30', 'cycle31', 'cycle32', 'cycle33', 'cycle34', 'cycle35', 'cycle36', 'cycle37', 'cycle38', 'cycle39'])


In [10]:
experiments = {
    'lr=0.001 wd=0.0005': 'results/lr0.001_wd0.0005',
    'lr=0.001 wd=0.005': 'results/lr0.001_wd0.005',
    'lr=0.001 wd=0.05': 'results/lr0.001_wd0.05',
    'lr=0.01 wd=0.0005': 'results/lr0.01_wd0.0005',
    'lr=0.01 wd=0.005': 'results/lr0.01_wd0.005',
    'lr=0.01 wd=0.05': 'results/lr0.01_wd0.05',
    'lr=0.1 wd=0.0005': 'results/lr0.1_wd0.0005',
    'lr=0.1 wd=0.005': 'results/lr0.1_wd0.005',
    'lr=0.1 wd=0.05': 'results/lr0.1_wd0.05',
}

all_results = {}
for exp_name, exp_path in tqdm(experiments.items()):
    all_results[exp_name] = get_experiments(Path(exp_path), 'seed*')


100%|██████████| 9/9 [00:26<00:00,  3.00s/it]


In [19]:
all_results['lr=0.001 wd=0.0005'][0]['cycle0'].keys()

dict_keys(['test_stats', 'labeled_indices', 'n_labeled_samples', 'unlabeled_indices', 'n_unlabeled_samples'])

In [16]:
learning_curves = {}

for exp_name in experiments:
    d = {}
    vals = [seed_results['test_stats']['accuracy'] for seed_results in all_results[exp_name]]
    d['accuracy'] = np.mean(vals)
    # d[metric_key + '_std'] = np.std(vals)
    result_dict[exp_name] = d
# print(pd.DataFrame(result_dict).T.to_markdown())

KeyError: 'test_stats'